# Notebook de reconocimiento de números escritos a mano con CNN.

In [1]:
using Pkg; Pkg.activate("/home/oscar/.julia/environments/CNN")

# Loading packages and data
# See: https://github.com/FluxML/model-zoo/blob/master/vision/mnist/conv.jl
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /home/oscar/.julia/packages/CUDAdrv/aBgcd/src/CUDAdrv.jl:69


In [4]:
# Load labels and images from Flux.Data.MNIST
# Train set: images used to estimate the CNN
# Load data on gpu (if enabled)
train_labels = MNIST.labels(:train);
train_imgs = MNIST.images(:train);

# Test set: images used to see how well the CNN perform "out-of-the-sample"
test_imgs = MNIST.images(:test)
test_labels = MNIST.labels(:test)

print("Images in the train set: $(size(train_imgs))")
print("Images in the test set: $(size(test_imgs))")

# Visualization of one digit
NROWS, NCOLS = 28, 28
a = reshape(train_imgs[1], NROWS, NCOLS)

Images in the train set: (60000,)Images in the test set: (10000,)

In [23]:
using Images, ImageFeatures # We have to Pkg.add("Netpbm") because our images are in pgm format

arc = 28

imgs = []
train_labels = []
path_train = "/home/oscar/Dropbox/JuliaCon/datasets/Yale/CroppedYale$(arc)/train/"
for directory = readdir(path_train)
    for file = readdir(path_train*directory)
        #println(path_train*directory*"/"*file, directory[end-1:end])
        push!(imgs, load(path_train*directory*"/"*file))
        label = Meta.parse(directory[end-1:end])
        if label < 14
            push!(train_labels, label)
        else
            push!(train_labels, label-1)
        end
    end
end
labels = onehotbatch(train_labels, 1:38);
# Partition into batches of size 64
train2 = [(cat(float.(imgs[i])..., dims = 4), labels[:,i])
         for i in partition(1:size(train_labels)[1], 64)];


In [25]:
imgs = MNIST.images(:train);
labels = onehotbatch(MNIST.labels(:train), 0:9);

# Partition into batches of size 1,000
train = [(cat(float.(imgs[i])..., dims = 4), labels[:,i])
         for i in partition(1:60_000, 1000)];

In [34]:
train[1:37]

37-element Array{Tuple{Array{Gray{Float32},4},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}},1}:
 ([Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); … ; Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0)]

[Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); … ; Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0)]

[Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); … ; Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0

In [26]:
train2

37-element Array{Tuple{Array{Gray{Float32},4},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}},1}:
 ([Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); … ; Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0)]

[Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); … ; Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0)]

[Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); … ; Gray{Float32}(0.0) Gray{Float32}(0.0) … Gray{Float32}(0.0) Gray{Float32}(0.0); Gray{Float32}(0

In [35]:
tX = cat(float.(MNIST.images(:test)[1:1000])..., dims = 4)
tY = onehotbatch(MNIST.labels(:test)[1:1000], 0:9)

10×1000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false  false  false   true  false  …  false  false   true  false  false
 false  false   true  false  false     false  false  false  false  false
 false   true  false  false  false      true   true  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false   true     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
  true  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false   true

In [8]:
# Model definition
# See: https://github.com/FluxML/model-zoo/blob/master/vision/mnist/conv.jl
model = Chain(
    # First convolution, operating upon a 28x28 image
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    MaxPool((2,2)), #maxpooling

    # Second convolution, operating upon a 14x14 image
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)), #maxpooling

    # Third convolution, operating upon a 7x7 image
    Conv((3, 3), 32=>32,pad=(1,1), relu),
    MaxPool((2,2)),

    # Reshape 3d tensor into a 2d one, at this point it should be (3, 3, 32, N)
    # which is where we get the 288 in the `Dense` layer below:
    x -> reshape(x, :, size(x, 4)),
    Dense(288, 10),

    # Softmax to get probabilities
    softmax,
)

# Load on gpu (if available)
model = gpu(model);

In [10]:
# Batching 
# See: https://github.com/FluxML/model-zoo/blob/master/vision/mnist/conv.jl
# Bundle images together with labels and group into minibatchess
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:9)
    return (X_batch, Y_batch)
end
# The CNN only "sees" 128 images at each training cycle:
batch_size = 128
mb_idxs = partition(1:length(train_imgs), batch_size)
# train set in the form of batches
train_set = [make_minibatch(train_imgs, train_labels, i) for i in mb_idxs];
# train set in one-go: used to calculate accuracy with the train set
train_set_full = make_minibatch(train_imgs, train_labels, 1:length(train_imgs));
# test set: to check we do not overfit the train data:
test_set = make_minibatch(test_imgs, test_labels, 1:length(test_imgs));

In [13]:
# Loss function
# See: https://github.com/FluxML/model-zoo/blob/master/vision/mnist/conv.jl
# `loss()` calculates the crossentropy loss between our prediction `y_hat`
function loss(x, y)
    # Add some noise to the image
    # we reduce the risk of overfitting the train sample by doing so:
    x_aug = x .+ 0.1f0*gpu(randn(eltype(x), size(x)))

    y_hat = model(x_aug)
    return crossentropy(y_hat, y)
end
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

# ADAM optimizer
opt = ADAM(0.001);

In [12]:
# Training loop
# See: https://github.com/FluxML/model-zoo/blob/master/vision/mnist/conv.jl
best_acc = 0.0
last_improvement = 0
accuracy_target = 0.97 #Set an accuracy target. When reached, we stop training.
max_epochs = 100 #Maximum
for epoch_idx in 1:100
    global best_acc, last_improvement
    # Train for a single epoch
    @which Flux.train!(loss, params(model), train_set, opt)

    # Calculate accuracy:
    acc = accuracy(train_set_full...)
    @info(@sprintf("[%d]: Train accuracy: %.4f", epoch_idx, acc))
    
    # Calculate accuracy:
    acc = accuracy(test_set...)
    @info(@sprintf("[%d]: Test accuracy: %.4f", epoch_idx, acc))

    # If our accuracy is good enough, quit out.
    if acc >= accuracy_target
        @info(" -> Early-exiting: We reached our target accuracy of $(accuracy_target*100)%")
        break
    end

    if epoch_idx - last_improvement >= 10
        @warn(" -> We're calling this converged.")
        break
    end
end

UndefVarError: UndefVarError: loss not defined

In [ ]:
# Get predictions and convert data to Array: 
pred = Tracker.data(model(test_set[1])); 

# Function to get the row index of the max value: 
f1(x) = getindex.(argmax(x, dims=1), 1) # Final predicted value is the one with the maximum probability: 
pred = f1(pred) .- 1; #minus 1, because the first digit is 0 (not 1)

In [ ]:
println("Predicted value = $(pred[1])")